In [1]:
import pandas as pd
import pickle
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
import re
import gensim
from skimage import io
import matplotlib.pyplot as plt
from gensim.test.utils import get_tmpfile

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd /content/drive/My Drive/vinepair_chatbot

/content/drive/My Drive/vinepair_chatbot


# Load Data

In [5]:
!sudo apt-get install python3-dev default-libmysqlclient-dev
!pip install pymysql
!pip install mysql-connector

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
The following NEW packages will be installed:
  default-libmysqlclient-dev libmysqlclient-dev
0 upgraded, 2 newly installed, 0 to remove and 6 not upgraded.
Need to get 995 kB of archives.
After this operation, 6,023 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmysqlclient-dev amd64 5.7.31-0ubuntu0.18.04.1 [992 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 default-libmysqlclient-dev amd64 1.0.4 [3,736 B]
Fetched 995 kB in 2s (458 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This fro

In [6]:
import pymysql
import pandas as pd
import numpy as np
import mysql.connector

db = mysql.connector.connect(
   host="database-3.cwonoul9fvix.us-east-1.rds.amazonaws.com",
   user="admin",
   passwd="password",
   database="VinePair"
)

print(db)

In [ ]:
db_cursor = db.cursor()
db_cursor.execute('''
create table cb_review_draft as 
(select p.ID, p.post_type, p.post_title, r.object_id, tax.term_taxonomy_id, tax.taxonomy,term.term_id, term.name, term.slug
from wp_posts p 
inner join wp_term_relationships r 
on p.ID = r.object_id 
inner join wp_term_taxonomy tax on 
r.term_taxonomy_id = tax.term_taxonomy_id
inner join wp_terms term on
tax.term_id = term.term_id
where post_type = "review"
and post_status = "publish")
''')
db_cursor.execute('''
delete from cb_review_draft where taxonomy = "author"
''')

db_cursor.execute('''
UPDATE cb_review_draft SET post_title = REPLACE(post_title, 'Review: ', '')
''')

db_cursor.execute('''
create table cb_review_draft2 as (SELECT post_title,
group_concat(name separator ',') as "names" 
FROM cb_review_draft
group by post_title)
''')

db_cursor.execute('''
DELETE FROM cb_review_draft2 where names like '%Beer%' or names like '%Spirit%'
''')

db_cursor.execute('SELECT * FROM cb_review_draft2')
table_rows = db_cursor.fetchall()
df_draft = pd.DataFrame(table_rows, columns =["title", "description"])


In [15]:
df_draft.to_pickle("data/wine_data.pkl")
df_draft

,title,description
0,14 Hands Winery Rosé,"Wine,Rosé,Wine,Syrah / Shiraz,Party Wine,Wine ..."
1,3 Badge Enology 'Leese-Fitch' Pinot Noir,"Wine,Pinot Noir,Wine,Pinot Noir,California,Par..."
2,"3 Badge Enology ""Plungerhead"" Unoaked Chardonnay","Wine,Chardonnay,Wine,Chardonnay,Picnics In The..."
3,3 Badge Enology Moobuzz Monterey Chardonnay,"Wine,Chardonnay,Wine,Chardonnay,Picnics In The..."
4,3 Badge Enology Moobuzz Pinot Noir,"Wine,Pinot Noir,Wine,Pinot Noir,Dinner Parties..."
...,...,...
1940,Zenato Valpolicella Superiore,"Wine,Red Blend,Wine,Impressing On A Budget,Cor..."
1941,Ziata Napa Valley Sauvignon Blanc,"Wine,Sauvignon Blanc,Wine,Sauvignon Blanc,Dinn..."
1942,Zind Humbrecht 'Hengst' Grand Cru Gewürztramin...,"Wine,Gewürztraminer,Wine,Gewürztraminer,Long-T..."
1943,Zonin Prosecco NV,"Wine,Prosecco,Sparkling Wine,Zonin,Wine,Brunch..."


In [ ]:
# db_cursor = db.cursor()
# db_cursor.execute('SELECT * FROM cb_review_draft2')
# table_rows = db_cursor.fetchall()
# df_draft = pd.DataFrame(table_rows, columns =["title", "description"])
# df_draft.to_pickle("data/wine_data.pkl")
# df_draft

,title,description
0,14 Hands Winery Rosé,"Wine,Rosé,Wine,Syrah / Shiraz,Party Wine,Wine ..."
1,3 Badge Enology 'Leese-Fitch' Pinot Noir,"Wine,Pinot Noir,Wine,Pinot Noir,California,Par..."
2,"3 Badge Enology ""Plungerhead"" Unoaked Chardonnay","Wine,Chardonnay,Wine,Chardonnay,Picnics In The..."
3,3 Badge Enology Moobuzz Monterey Chardonnay,"Wine,Chardonnay,Wine,Chardonnay,Picnics In The..."
4,3 Badge Enology Moobuzz Pinot Noir,"Wine,Pinot Noir,Wine,Pinot Noir,Dinner Parties..."
...,...,...
1940,Zenato Valpolicella Superiore,"Wine,Red Blend,Wine,Impressing On A Budget,Cor..."
1941,Ziata Napa Valley Sauvignon Blanc,"Wine,Sauvignon Blanc,Wine,Sauvignon Blanc,Dinn..."
1942,Zind Humbrecht 'Hengst' Grand Cru Gewürztramin...,"Wine,Gewürztraminer,Wine,Gewürztraminer,Long-T..."
1943,Zonin Prosecco NV,"Wine,Prosecco,Sparkling Wine,Zonin,Wine,Brunch..."


In [ ]:
# Load description features
df = pd.read_pickle("data/wine_data.pkl")
print(df.shape)
df.head(3)

(1945, 2)


,title,description
0,14 Hands Winery Rosé,"Wine,Rosé,Wine,Syrah / Shiraz,Party Wine,Wine ..."
1,3 Badge Enology 'Leese-Fitch' Pinot Noir,"Wine,Pinot Noir,Wine,Pinot Noir,California,Par..."
2,"3 Badge Enology ""Plungerhead"" Unoaked Chardonnay","Wine,Chardonnay,Wine,Chardonnay,Picnics In The..."


## Clean text

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def stem_words(text):
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

In [ ]:
df['description'] = df.description.apply(func=make_lower_case)
df['description'] = df.description.apply(func=remove_stop_words)
df['description'] = df.description.apply(func=remove_punctuation)
df['description'] = df.description.apply(func=stem_words)

In [ ]:
df.head(1)

,title,description
0,14 Hands Winery Rosé,wine rosé wine syrah shiraz parti wine wine co...


# TF-IDF Model

In [ ]:
df['full_document'] = df['description']

In [ ]:
#Fit TFIDF 
#Learn vocabulary and tfidf from all style_ids.
tf = TfidfVectorizer(analyzer='word', 
                     min_df=10,
                     ngram_range=(1, 2),
                     #max_features=1000,
                     stop_words='english')
tf.fit(df['full_document'])

#Transform style_id products to document-term matrix.
tfidf_matrix = tf.transform(df['full_document'])
pickle.dump(tf, open("models/tfidf_model.pkl", "wb"))

print(tfidf_matrix.shape)

(1945, 615)


In [ ]:
# Compress with SVD
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=500)
latent_matrix = svd.fit_transform(tfidf_matrix)
pickle.dump(svd, open("models/svd_model.pkl", "wb"))

print(latent_matrix.shape)

(1945, 500)


In [ ]:
n = 25 #pick components
#Use elbow and cumulative plot to pick number of components. 
#Need high ammount of variance explained. 
doc_labels = df.title
svd_feature_matrix = pd.DataFrame(latent_matrix[:,0:n] ,index=doc_labels)
print(svd_feature_matrix.shape)
svd_feature_matrix.head()

pickle.dump(svd_feature_matrix, open("models/lsa_embeddings.pkl", "wb"))

(1945, 25)


# Doc2Vec Model

Doc to vec preverves word order in the embeddings, so "I hate rose" and "I love rose" will be treated differently.

In [ ]:
#Use descriptions for vocabulary 
descriptions = df.description.values.tolist()

documents = []
for i in range(len(df)):
    mystr = descriptions[i]

    documents.append(re.sub("[^\w]", " ",  mystr).split())

In [ ]:
print(len(df))
print(len(documents))

1945
1945


In [ ]:
df.loc[278]

title                              Carmel Road Monterey Chardonnay
description      wine chardonnay wine chardonnay last minut win...
full_document    wine chardonnay wine chardonnay last minut win...
Name: 278, dtype: object

In [ ]:
formatted_documents = [gensim.models.doc2vec.TaggedDocument(doc, [i]) for i, doc in enumerate(documents)]

model = gensim.models.doc2vec.Doc2Vec(vector_size=25, min_count=5, epochs=200, seed=0, window=3, dm=1)
model.build_vocab(formatted_documents)

In [ ]:
%time model.train(formatted_documents, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 42.4 s, sys: 11 s, total: 53.4 s
Wall time: 40.2 s


In [ ]:
fname = get_tmpfile("models/doc2vec_model")
model.save("models/doc2vec_model")
model = gensim.models.doc2vec.Doc2Vec.load("./models/doc2vec_model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
vector = model.infer_vector(doc_words=["this", "is", "a", "test"], epochs=50)
vector

array([ 5.0727867e-02,  9.6120298e-02, -1.4574610e-01,  3.6759529e-02,
        1.3729359e-01, -1.0205120e-01,  6.6395128e-01, -4.5419267e-01,
        5.4041788e-02, -2.1587947e-02,  1.6775812e-01, -1.0572556e-01,
       -1.0948350e-01, -6.5739430e-04, -2.3674081e-01, -2.6343617e-01,
        3.1333558e-02, -2.7847102e-01,  5.4739442e-02,  5.3819199e-03,
       -1.5129122e-01,  4.7619894e-01,  2.0393278e-01,  2.4418649e-01,
       -4.7287505e-04], dtype=float32)

In [ ]:
doctovec_feature_matrix = pd.DataFrame(model.docvecs.vectors_docs, index=df.title)
print(doctovec_feature_matrix.shape)
doctovec_feature_matrix.head(3)
pickle.dump(doctovec_feature_matrix, open("models/doctovec_embeddings.pkl", "wb"))

(1945, 25)
